# AlexNet
이 노트북은 AlexNet을 구현하는 노트북이다.


In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [8]:
# 데이터셋 불러오기
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# 이름 설정하기
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# 데이터 split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 결과 확인
print("x_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)

x_train shape:  (40000, 32, 32, 3)
y_train shape:  (40000, 1)


In [9]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))

print("train_ds: ", train_ds)
print("test_ds: ", test_ds)
print("val_ds: ", val_ds)

train_ds:  <_TensorSliceDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))>
test_ds:  <_TensorSliceDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))>
val_ds:  <_TensorSliceDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1,), dtype=tf.uint8, name=None))>


In [10]:
def process_images(image, label):

    # Resize images form 32*32 to 227*227
    image = tf.image.resize(image, (227, 227))
    return image, label

In [11]:
train_ds = train_ds.batch(128, drop_remainder=True)
test_ds = test_ds.batch(128, drop_remainder=True)
val_ds = val_ds.batch(128, drop_remainder=True)

train_ds = train_ds.map(process_images)
test_ds = test_ds.map(process_images)
val_ds = val_ds.map(process_images)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

class AlexNet(Sequential):
    def __init__(self):
        super().__init__()

        self.add(Conv2D(96, kernel_size=11, strides=4, activation='relu', input_shape=(227, 227, 3)))
        self.add(BatchNormalization())
        self.add(MaxPool2D(3, 2))

        self.add(Conv2D(256, kernel_size=5, strides=1, activation='relu', padding='same'))
        self.add(BatchNormalization())
        self.add(MarkdownPool2D(3, 2))

        self.add(Conv2D(384, kernel_size=3, strides=1, activation='relu', padding='same'))
        self.add(BatchNormalization())

        self.add(Conv2D(384, kernel_size=3, strides=1, activation='relu', padding='same'))
        self.add(BatchNormalization())

        self.add(Conv2D(256, kernel_size=3, strides=1, activation='relu', padding='same'))
        self.add(BatchNormalization())
        self.add(MaxPool2D(3, 2))

        self.add(Flatten())
        self.add(Dense(4096, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(4096, activation='relu'))
        self.add(Dropout(0.5))
        self.add(Dense(10, activation='softmax'))

        self.complie(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])